In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd

# Set credentials 
os.environ['PG_HOST'] = 'isba-dev-02.cmb4w8cmqb26.us-east-1.rds.amazonaws.com'
os.environ['PG_USER'] = 'postgres'
os.environ['PG_PASSWORD'] = 'isba_4715'
os.environ['PG_DB'] = 'data_engineer_project'

host = os.getenv('PG_HOST')
user = os.getenv('PG_USER')
password = os.getenv('PG_PASSWORD')
database = os.getenv('PG_DB')

connection_string = f"postgresql+psycopg2://{user}:{password}@{host}/{database}"
engine = create_engine(connection_string)
pd.set_option('display.max_rows', None)

test_query = '''
SELECT COUNT(*) FROM raw."Gold_Ranked_Players"
limit 10;
'''
pd.read_sql(test_query, engine)

,count
0,820


# Descriptive Business Question: What are the win/loss ratios for new players and experienced players?

In [3]:
sql_query = '''
WITH player_types AS (
    SELECT
        CASE
            WHEN "freshBlood" = 'TRUE' THEN 'New Player'
            ELSE 'Experienced Player'
        END AS player_type,
        wins,
        losses
    FROM raw."Gold_Ranked_Players"
)
SELECT
    player_type,
    COUNT(*) AS player_count,
    ROUND(AVG(wins)::numeric / NULLIF(AVG(losses), 0), 2) AS win_loss_ratio
FROM player_types
GROUP BY player_type
ORDER BY player_type;
'''


# Execute
query_result = pd.read_sql(sql_query, engine)
query_result

,player_type,player_count,win_loss_ratio
0,Experienced Player,760,1.04
1,New Player,60,1.02


# Insights: 
1.New players (freshBlood = TRUE) had a win/loss ratio of 1.02
2.Experienced players had a slightly higher ratio of 1.04
3.Despite having significantly less players,new players are not at a major disadvantage competitively. 
4.Indication of skill-based matchmaking, new players will not lose more just because they lack game experience. 

# Recommendations
1.

In [ ]:


# Execute
query_result = pd.read_sql(sql_query, engine)
query_result

,hotStreak,player_count,avg_win_loss_ratio
0,TRUE,13,2.39
1,FALSE,47,0.99
